In [ ]:
import tensorflow as tf
import os
import shutil
import zipfile
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical # Import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For image visualizations (if needed)
from sklearn.metrics import confusion_matrix  # For evaluating model performance

In [ ]:
datasets = "/content/drive/MyDrive/dataset.zip"
target_dir = 'data'  # Extract into Data Folder
os.makedirs(target_dir, exist_ok=True)

with zipfile.ZipFile(datasets, 'r') as zip_ref:
    zip_ref.extractall(target_dir)

In [ ]:
data_dir = 'data/'  # Replace with the actual path to your dataset folder
target_size = (224, 224)  # Size for images; adjust based on your chosen model
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalization for pixel values
    rotation_range=30,  # Data augmentation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'  # Since we have multiple classes
)

Found 21921 images belonging to 3 classes.


In [ ]:
# No augmentations for the test data
test_datagen = ImageDataGenerator(rescale=1./255)

def custom_flow(data_dir, target_size, batch_size, class_mode, num_samples_per_class):
    class_folders = os.listdir(data_dir)
    all_images = []
    all_labels = []

    for i, class_folder in enumerate(class_folders):
        class_path = os.path.join(data_dir, class_folder)

        for subfolder in os.listdir(class_path):  # Iterate through subfolders
            subfolder_path = os.path.join(class_path, subfolder)
            image_names = os.listdir(subfolder_path)[:num_samples_per_class]

            for image_name in image_names:
                img_path = os.path.join(subfolder_path, image_name)
                if os.path.isfile(img_path):
                    img = load_img(img_path, target_size=target_size)
                    img_array = img_to_array(img)
                    all_images.append(img_array)
                    all_labels.append(i)

    # Convert labels to categorical format
    all_images = np.array(all_images)
    all_labels = np.array(all_labels)
    all_labels = to_categorical(all_labels, num_classes=len(class_folders))

    return all_images, all_labels

In [ ]:
num_samples_per_class = 1000

In [ ]:
# Generate test data using the custom flow function
test_images, test_labels = custom_flow(data_dir, target_size, batch_size, 'categorical', num_samples_per_class)

# Check the number of test samples generated
print("Number of test samples:", test_images.shape[0])

Number of test samples: 6000


In [ ]:
# Model Definition
base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=target_size + (3,))

# Freeze base model layers (optional, for initial training)
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)

# Disease Classifier
x = Dense(128, activation='relu')(x)
disease_predictions = Dense(3, activation='softmax', name='disease_predictions')(x)

# Disease-Specific Branches (example)
pneumonia_branch = Dense(32, activation='relu')(x)
pneumonia_output = Dense(1, activation='sigmoid', name='pneumonia_output')(pneumonia_branch)

malaria_branch = Dense(32, activation='relu')(x)
malaria_output = Dense(1, activation='sigmoid', name='malaria_output')(malaria_branch)

breastcancer_branch=Dense(32, activation='relu')(x)
breastcancer_output=Dense(1, activation='sigmoid', name='breastcancer_output')(breastcancer_branch)


model = tf.keras.Model(inputs=base_model.input,
                       outputs=[disease_predictions, pneumonia_output, malaria_output, breastcancer_output])



94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
# Compilation
model.compile(optimizer='adam',
              loss={'disease_predictions': 'categorical_crossentropy',
                    'pneumonia_output': 'binary_crossentropy',
                    'malaria_output': 'binary_crossentropy',
                    'breastcancer_output': 'binary_crossentropy'
                   },
              metrics=['accuracy'])

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255) # May not need augmentation here

validation_data = val_datagen.flow_from_directory(
    data_dir,  # Assuming a separate validation subfolder within 'dataset'
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 21921 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a checkpoint callback to save the model weights
checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/saved_model.h5',  # Specify the file path where the model will be saved
    monitor='val_loss',  # Monitor validation loss for saving the best model
    save_best_only=True,  # Save only the best model based on validation loss
    mode='min',  # Save the model with the lowest validation loss
    verbose=1  # Show messages when saving the model
)

In [ ]:
history = model.fit(
    train_data,
    epochs=6,  # Adjust the number of epochs
    validation_data=validation_data,
    callbacks=[checkpoint_callback]
)


Epoch 1/6
172/172 [==============================] - ETA: 0s - loss: 3.1965 - disease_predictions_loss: 1.0493 - pneumonia_output_loss: 0.7292 - malaria_output_loss: 0.6858 - breastcancer_output_loss: 0.7322 - disease_predictions_accuracy: 0.8274 - pneumonia_output_accuracy: 0.6530 - malaria_output_accuracy: 0.6504 - breastcancer_output_accuracy: 0.6397 
Epoch 1: val_loss improved from inf to 2.02840, saving model to /content/drive/MyDrive/saved_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


172/172 [==============================] - 8685s 51s/step - loss: 3.1965 - disease_predictions_loss: 1.0493 - pneumonia_output_loss: 0.7292 - malaria_output_loss: 0.6858 - breastcancer_output_loss: 0.7322 - disease_predictions_accuracy: 0.8274 - pneumonia_output_accuracy: 0.6530 - malaria_output_accuracy: 0.6504 - breastcancer_output_accuracy: 0.6397 - val_loss: 2.0284 - val_disease_predictions_loss: 0.1037 - val_pneumonia_output_loss: 0.6395 - val_malaria_output_loss: 0.6382 - val_breastcancer_output_loss: 0.6469 - val_disease_predictions_accuracy: 0.9952 - val_pneumonia_output_accuracy: 0.6666 - val_malaria_output_accuracy: 0.6667 - val_breastcancer_output_accuracy: 0.6666
Epoch 2/6
172/172 [==============================] - ETA: 0s - loss: 2.0820 - disease_predictions_loss: 0.1535 - pneumonia_output_loss: 0.6394 - malaria_output_loss: 0.6466 - breastcancer_output_loss: 0.6425 - disease_predictions_accuracy: 0.9674 - pneumonia_output_accuracy: 0.6654 - malaria_output_accuracy: 0.6608

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('/content/drive/MyDrive/saved_model.h5')  # Replace 'saved_model.h5' with the actual file path

In [ ]:
# Import confusion_matrix function
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Generate predictions for the test data
y_pred = loaded_model.predict(test_data)

# Get true labels (convert one-hot encoding to class indices)
y_true = test_data.classes

# Calculate confusion matrices for each task
cm_disease = confusion_matrix(y_true, np.argmax(y_pred[0], axis=1))
cm_pneumonia = confusion_matrix(y_true, np.round(y_pred[1]))
cm_malaria = confusion_matrix(y_true, np.round(y_pred[2]))
cm_breastcancer = confusion_matrix(y_true, np.round(y_pred[3]))

# Plot confusion matrices using scikit-learn's ConfusionMatrixDisplay
fig, axes = plt.subplots(2, 2, figsize=(10, 10))  # Create a 2x2 grid for four matrices
fig.suptitle('Confusion Matrices')

disp_disease = ConfusionMatrixDisplay(cm_disease, display_labels=train_data.class_indices)
disp_disease.plot(ax=axes[0, 0]).set_title('Disease Classification')

disp_pneumonia = ConfusionMatrixDisplay(cm_pneumonia, display_labels=['Negative', 'Positive'])
disp_pneumonia.plot(ax=axes[0, 1]).set_title('Pneumonia')

disp_malaria = ConfusionMatrixDisplay(cm_malaria, display_labels=['Negative', 'Positive'])
disp_malaria.plot(ax=axes[1, 0]).set_title('Malaria')

disp_breastcancer = ConfusionMatrixDisplay(cm_breastcancer, display_labels=['Negative', 'Positive'])
disp_breastcancer.plot(ax=axes[1, 1]).set_title('Breast Cancer')

plt.tight_layout()
plt.show()

 10/686 [..............................] - ETA: 1:12:59

KeyboardInterrupt: 